## Modelo de datos de biblioteca de música

Este modelo de datos representa una biblioteca de música que tiene una gran colección de canciones y muestra sus canciones más descargadas casi en tiempo real.

Los patrones de acceso que este modelo de datos facilita son:

* Recuperación de un disco de canción, facilitada por una tabla llamada Songs.

* Recuperación de un registro de descarga específico o todos los registros de descarga de una canción, facilitada por una tabla llamada Songs.

* Recuperación de un registro de recuento mensual específico de descargas o todos los registros de recuento de descargas mensuales de una canción, facilitada por una tabla llamada Song.

* Recuperación de todos los registros (incluidos los registros de canciones, los registros de descargas y los registros de recuento de descargas mensuales) de una canción, facilitada por una tabla llamada Songs.

* Buscar la mayoría de las canciones descargadas, facilitado por el índice secundario global de la tabla Canciones llamado DownloadsByMonth.

In [1]:
import pandas as pd
import dynamodb
from dynamodb import DynamoTable
import json
from decimal import Decimal

In [2]:
dt = DynamoTable()
try:
    dt.select_table('Songs-c')
    print(dt)
except:
    dt.create_table(
        table_name='Songs-c',
        partition_key='Id',
        partition_key_type='S',
        sort_key='Metadata',
        sort_key_type='S',
        provisioned=False
    )

- Table name: Songs-c            
- Table arn: arn:aws:dynamodb:us-east-1:621304841877:table/Songs-c            
- Table creation: 2023-03-06 21:09:05.911000-03:00            
- [{'AttributeName': 'Id', 'KeyType': 'HASH'}, {'AttributeName': 'Metadata', 'KeyType': 'RANGE'}]            
- [{'AttributeName': 'DownloadMonth', 'AttributeType': 'S'}, {'AttributeName': 'Id', 'AttributeType': 'S'}, {'AttributeName': 'Metadata', 'AttributeType': 'S'}, {'AttributeName': 'TotalDownloadsInMonth', 'AttributeType': 'N'}]


In [ ]:
# Itireate over json file and add items to table
with open('songs.json') as json_file:
    songs = json.load(json_file)
songs_form = json.loads(json.dumps(songs), parse_float=Decimal)

with dt.table.batch_writer() as batch:
    for song in songs_form:
        batch.put_item(Item=song)

In [ ]:
df = dt.query_items(query="1", to_pandas=True, consumed_capacity=True)
df

* DynamoDB permite la creación de índices secundarios para obtener más patrones de acceso a los datos en la tabla. Los índices secundarios son un medio eficaz para agregar flexibilidad de consulta a la tabla de DynamoDB.

* DynamoDB tiene dos tipos de índices secundarios: los índices secundarios globales y los índices secundarios locales. En esta sección, se agrega un índice secundario global al atributo de categoría.

In [6]:
dt.create_global_secondary_index(
    att_name="Artist",
    att_type="S",
    i_name="ArtistNameIndex",
    proj_type=["Title"],
    provisioned=False,
    i_rcu=5,
    i_wcu=5
)

{'TableDescription': {'AttributeDefinitions': [{'AttributeName': 'Artist',
    'AttributeType': 'S'},
   {'AttributeName': 'Id', 'AttributeType': 'S'},
   {'AttributeName': 'Metadata', 'AttributeType': 'S'}],
  'TableName': 'Songs-c',
  'KeySchema': [{'AttributeName': 'Id', 'KeyType': 'HASH'},
   {'AttributeName': 'Metadata', 'KeyType': 'RANGE'}],
  'TableStatus': 'UPDATING',
  'CreationDateTime': datetime.datetime(2023, 3, 6, 21, 9, 5, 911000, tzinfo=tzlocal()),
  'ProvisionedThroughput': {'NumberOfDecreasesToday': 0,
   'ReadCapacityUnits': 0,
   'WriteCapacityUnits': 0},
  'TableSizeBytes': 0,
  'ItemCount': 0,
  'TableArn': 'arn:aws:dynamodb:us-east-1:621304841877:table/Songs-c',
  'TableId': '6bac4845-e184-40dd-89a2-002bbedc4e34',
  'BillingModeSummary': {'BillingMode': 'PAY_PER_REQUEST',
   'LastUpdateToPayPerRequestDateTime': datetime.datetime(2023, 3, 6, 21, 27, 50, 12000, tzinfo=tzlocal())},
  'GlobalSecondaryIndexes': [{'IndexName': 'ArtistNameIndex',
    'KeySchema': [{'Attr

In [13]:
dt.create_global_secondary_index(
    att_name="DownloadMonth",
    att_type="S",
    sort_index="TotalDownloadsInMonth",
    sort_type="N",
    i_name="DownloadsByMonth",
    provisioned=False,
    i_rcu=1,
    i_wcu=1
)

Global secondary index could not be created in table Songs-c. Here's why: ValidationException: Attempting to create an index which already exists


ClientError: An error occurred (ValidationException) when calling the UpdateTable operation: Attempting to create an index which already exists

Crear un índice secundario global es muy parecido a crear una tabla. Se especifican el nombre del índice, los atributos que estarán dentro del índice, el esquema de la clave del índice y el rendimiento aprovisionado (la capacidad máxima que una aplicación puede consumir de una tabla o de un índice). El rendimiento aprovisionado en cada índice es independiente del rendimiento aprovisionado en una tabla. Esto permite definir el rendimiento con detalle a fin de satisfacer las necesidades de la aplicación.

In [15]:
dt.check_status_gsi()

'ACTIVE'